<a href="https://colab.research.google.com/github/antonellagambarte/procesamiento_del_lenguaje_I/blob/main/desafio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. tf o por conteo (para vectorizador).
2. clasificacion por prototipo. de test, miro train, quien es el mas cercano(en base a similaridad coseno) y copio etiqueta. Miro quien tengo mas cerca. calcular f1 macro. Agarro cada elemento del test y buscar el mas cercano del train.
4. tomo la generada con las distintas representaciones de los documentos.

In [108]:
%pip install numpy scikit-learn

In [109]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from collections import Counter

### **Punto 1**

In [110]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

Vectorización:

In [111]:

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [112]:
np.random.seed(42)
idxs = np.random.choice(len(newsgroups_train.data), 5, replace=False)

Función para mostrar el contenido de los documentos, ya sea completo o solo los primeros 800 caracteres.

In [113]:
def show_document(idx,i,show_complete = False):
    print("="*80)
    print(f"Documento {i+1} (índice {idx}) - Clase: \033[1m{newsgroups_train.target_names[newsgroups_train.target[idx]]}\033[0m")
    if show_complete:
        print(newsgroups_train.data[idx])
    else:
        print(newsgroups_train.data[idx][:800], "...\n")

Muestro los 5 textos elegidos aleatoriamente:

In [114]:
for i, idx in enumerate(idxs):
    show_document(idx,i)


Documento 1 (índice 7492) - Clase: comp.sys.mac.hardware
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...

Documento 2 (índice 3546) - Clase: comp.os.ms-windows.misc


     Don't bother if you have CPBackup or Fastback.  They all offer options 
not available in the stripped-down MS version (FROM CPS!).  Examples - no 
proprietary format (to save space), probably no direct DMA access, and no 
tape drive! ...

Documento 3 (índice 5582) - Clase: misc.forsale
5.25" Internal Low density disk drive.

Monochrome monitor

8088 motherboard, built in parallel and serial ports, built in mono and
color output, 7Mhz.

Libertarian, atheist, semi-anarchal Techno-Rat. ...

Documento 4 (índice 4793) - Clase: talk.politics.guns
Hi,

In Canada, any gun that enters a Natio

Función para cálculo de similitud y muestra de resultados:

In [115]:
def show_similar_documents(idx,i, X_train, newsgroups_train, show_complete=False):

    print("\nDocumento original.\n")
    show_document(idx, i, show_complete)

    # Calculo de similitudes
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    most_similar = np.argsort(cossim)[::-1][1:6]  # Se obtienen los 5 más similares.

    print("\n5 documentos más similares:\n")

    for k, j in enumerate(most_similar, 0):
        print(f" - idx {j}, clase: \033[1m{newsgroups_train.target_names[newsgroups_train.target[j]]}\033[0m, similitud: \033[1m{cossim[j]:.3f}\033[0m")
        show_document(j, k, show_complete)



Documento 1:

In [116]:
idx_interes = idxs[0]
show_similar_documents(idx_interes,0, X_train, newsgroups_train)


Documento original.

Documento 1 (índice 7492) - Clase: comp.sys.mac.hardware
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...


5 documentos más similares:

 - idx 10935, clase: comp.sys.mac.hardware, similitud: 0.667
Documento 1 (índice 10935) - Clase: comp.sys.mac.hardware
Hey everybody:

   I want to buy a mac and I want to get a good price...who doesn't?  So,
could anyone out there who has found a really good deal on a Centris 650
send me the price.  I don't want to know where, unless it is mail order or
areound cleveland, Ohio.  Also, should I buy now or wait for the Power PC.

Thanks.
BoB
reply via post or e-mail at rrn@po.cwru.edu
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is s

El primer documento corresponde a la etiqueta: comp.sys.mac.hardware.

Comparando con los 5 más similares se encuentra que los documentos 1,2,3 y 5 corresponden a la misma etiqueta, lo que da sentido a la similitud encontrada.

En cuanto al documento 4, corresponde a la etiqueta misc.forsale. Mirando su contenido, se trata de un anuncio de venta de un producto electrónico, por lo que tiene sentido la similitud.

Documento 2:

In [117]:
idx_interes = idxs[1]
show_similar_documents(idx_interes,1, X_train, newsgroups_train, show_complete= True)


Documento original.

Documento 2 (índice 3546) - Clase: comp.os.ms-windows.misc


     Don't bother if you have CPBackup or Fastback.  They all offer options 
not available in the stripped-down MS version (FROM CPS!).  Examples - no 
proprietary format (to save space), probably no direct DMA access, and no 
tape drive!

5 documentos más similares:

 - idx 5665, clase: comp.sys.ibm.pc.hardware, similitud: 0.204
Documento 1 (índice 5665) - Clase: comp.sys.ibm.pc.hardware

By initiating a DMA xfer.  :)

Seriously, busmastering adapter have their own DMA ability, they don't use
the motherboards on-board DMA(which is *MUCH* slower).

ISA has no bus arbitration, so if two busmastering cards in 1 ISA system
try to do DMA xfers on the same DMA channel the system will lock or 
crash.(I forget)

Their are 8 DMA channels in an ISA system. 0-7. 0-3 are 8-bit & 4-7 are
16-bit.

The system uses DMA 0, a SoundBlaster uses DMA 1.

I could buy a busmastering XGA-2 video card & a busmastering SCSI HA.


El segundo documento corresponde a la etiqueta: comp.os.ms-windows.misc.

Analizando los 5 más similares se encuentra que todos corresponden a la etiqueta comp.sys.ibm.pc.hardware. Al revisar el contenido completo de cada uno, se puede notar que todos tratan sobre el uso del DMA (Acceso Directo a Memoria) en sistemas antiguos de hardware, que es la temática del segundo documento, por lo que tiene sentido la similitud con todos ellos.

Documento 3:

In [118]:
idx_interes = idxs[2]
show_similar_documents(idx_interes,2, X_train, newsgroups_train, show_complete= True)


Documento original.

Documento 3 (índice 5582) - Clase: misc.forsale
5.25" Internal Low density disk drive.

Monochrome monitor

8088 motherboard, built in parallel and serial ports, built in mono and
color output, 7Mhz.

Libertarian, atheist, semi-anarchal Techno-Rat.

5 documentos más similares:

 - idx 5510, clase: misc.forsale, similitud: 0.462
Documento 1 (índice 5510) - Clase: misc.forsale
I am looking for a 286 motherboard, preferable 12 or 16, 640k or 1 meg RAM. 
I am also looking for a VGA card.

Am willing to trade 1200 external, 5.25" LD Drive, 8088 motherboard,
monochrome monitor, Game Boy, in some combination for the above.

Libertarian, atheist, semi-anarchal Techno-Rat.
 - idx 4922, clase: misc.forsale, similitud: 0.300
Documento 2 (índice 4922) - Clase: misc.forsale
For sale:

Nintendo Game Boy, Tetris, Castlevania Adventure, All-Star Challenge,
Nemesis, Play-Action football, link cable.

Make me an offer.

Libertarian, atheist, semi-anarchal Techno-Rat.
 - idx 4347, c

El tercer documento corresponde a la etiqueta: misc.forsale.

Comparando con los 5 más similares se encuentra que los documentos 1,2 y 4 corresponden a la misma etiqueta, lo que da sentido a la similitud encontrada.

En cuanto a los documentos 3 y 5, corresponden a la etiqueta comp.graphics. Se observa que tratan sobre tecnología de computadoras antiguas y su uso en tareas como gráficos 3D, mpeg, y aplicaciones relacionadas con la gráfica por software y hardware de pantalla. Tiene en común por ejemplo monitor o la firma "Libertarian, atheist, semi-anarchal Techno-Rat", lo que puede dar sentido a la similitud encontrada ya que tratan sobre los mismos temas.

Documento 4:

In [119]:
idx_interes = idxs[3]
show_similar_documents(idx_interes,3, X_train, newsgroups_train, show_complete= True)


Documento original.

Documento 4 (índice 4793) - Clase: talk.politics.guns
Hi,

In Canada, any gun that enters a National Park must be sealed (I think it's a
small metal tag that's placed over the trigger).  The net result of this is
that you _can't_ use a gun to protect yourself from bears (or psychos) in the
National Parks.  Instead, one has to be sensitive to the dangers and annoyances
of hiking in bear country, and take the appropriate precautions.

I think this policy makes the users of the National Parks feel a little closer
to Nature, that they are a part of Nature and, as such, have to deal with
nature on it's own terms.

5 documentos más similares:

 - idx 6894, clase: talk.politics.guns, similitud: 0.236
Documento 1 (índice 6894) - Clase: talk.politics.guns
Here is a press release from the White House.

 President Clinton's Remarks On Waco With Q/A
 To: National Desk
 Contact: White House Office of the Press Secretary, 202-456-2100

   WASHINGTON, April 20 -- Following are r

El cuarto documento corresponde a la etiqueta: talk.politics.guns.

Comparando con los 5 más similares se encuentra que los documentos 1 y 4 corresponden a la misma etiqueta, lo que da sentido a la similitud encontrada, ya que se trataría de temas similares.

En cuanto a los documentos 3 y 5, corresponden a la etiqueta comp.graphics. Se observa que tratan sobre tecnología de computadoras antiguas y su uso en tareas como gráficos 3D, mpeg, y aplicaciones relacionadas con la gráfica por software y hardware de pantalla. Tiene en común por ejemplo monitor o la firma "Libertarian, atheist, semi-anarchal Techno-Rat", lo que puede dar sentido a la similitud encontrada ya que tratan sobre los mismos temas.

In [120]:
for i, idx in enumerate(idxs):
    show_document(idx,i)
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    most_similar = np.argsort(cossim)[::-1][1:6]
    print("5 documentos más similares:")
    for j in most_similar:
        print(f" - idx {j}, clase: {newsgroups_train.target_names[newsgroups_train.target[j]]}, similitud: {cossim[j]:.3f}")
    print()

Documento 1 (índice 7492) - Clase: comp.sys.mac.hardware
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...

5 documentos más similares:
 - idx 10935, clase: comp.sys.mac.hardware, similitud: 0.667
 - idx 7258, clase: comp.sys.ibm.pc.hardware, similitud: 0.348
 - idx 4971, clase: comp.sys.mac.hardware, similitud: 0.180
 - idx 4303, clase: misc.forsale, similitud: 0.155
 - idx 645, clase: comp.sys.mac.hardware, similitud: 0.141

Documento 2 (índice 3546) - Clase: comp.os.ms-windows.misc


     Don't bother if you have CPBackup or Fastback.  They all offer options 
not available in the stripped-down MS version (FROM CPS!).  Examples - no 
proprietary format (to save space), probably no direct DMA access, and no 
tape drive! ...

5 documentos más similares:


### **Punto 2**

In [121]:

X_test = tfidfvect.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Calculo de matriz de similitud entre test y train
sim_matrix = cosine_similarity(X_test, X_train)

# Para cada documento de test, se obtiene el índice del documento de train más similar
most_similar_idx = np.argmax(sim_matrix, axis=1)

# Predice usando la clase del documento más similar
y_pred_proto = y_train[most_similar_idx]

# Evaluación con F1 macro
score = f1_score(y_test, y_pred_proto, average='macro')
print("F1-score macro (prototipo):", score)

# Muestra de algunos de los documentos etiquetados
np.random.seed(0)
sample_idxs = np.random.choice(len(newsgroups_test.data), 3, replace=False)

for i, idx in enumerate(sample_idxs):
    true_class = newsgroups_test.target_names[y_test[idx]]
    pred_class = newsgroups_train.target_names[y_pred_proto[idx]]
    most_sim_idx = most_similar_idx[idx]

    print("\n" + "="*100)
    print(f"Documento {i+1} (índice test={idx})")
    print(f"Clase real:     \033[1m{true_class}\033[0m")
    print(f"Clase predicha: \033[1m{pred_class}\033[0m")
    print(f"Documento más similar: train idx={most_sim_idx} ({newsgroups_train.target_names[y_train[most_sim_idx]]})")

    print("\n--- Documento de TEST ---")
    print("="*80)
    print(f"Documento test {i+1} (índice {idx}) - Clase: \033[1m{true_class}\033[0m")
    print(newsgroups_test.data[idx][:800], "...\n")

    print("--- Documento más similar del TRAIN ---")
    show_document(most_sim_idx, i)


F1-score macro (prototipo): 0.5049911553681621

Documento 1 (índice test=3856)
Clase real:     soc.religion.christian
Clase predicha: soc.religion.christian
Documento más similar: train idx=7847 (soc.religion.christian)

--- Documento de TEST ---
Documento test 1 (índice 3856) - Clase: soc.religion.christian
David Hammerslag asked:

	How do you (Mormons) reconcile the idea of eternal marriage
	with Christ's statement that in the resurrection people will
	neither marry nor be given in marriage (Luke, chapt. 20)?

Several explanations for this seeming contradiction have been proposed,
but most LDS scholars whose opinions I have studied take more or less
one of the following three positions:

(1) Jesus was talking to a group of people (Sadducees) who were trying
    to trip Him up with what they felt was a silly hypothetical situa-
    tion that ridiculed the concept of a resurrection (something they
    didn't believe in).  These people -- and those associated with them
    ("Now there w

Se muestran 3 ejemplos elegidos al azar. En el primero y en el tercero se observa que asignó una etiqueta idéntica a la original. Sin embargo, en el segundo documento, la etiqueta original era sci.crypt. En la clasificación, se le asignó la etiqueta talk.politics.mideast.

Analizando el contenido de ambos, se tratan de textos que hablan de temas totalmente diferentes.

In [122]:
# Ver la distribución de las etiquetas
class_distribution = Counter(y_train)
class_names = newsgroups_train.target_names
for label, count in class_distribution.items():
    print(f"{class_names[label]}: {count}")


rec.autos: 594
comp.sys.mac.hardware: 578
comp.graphics: 584
sci.space: 593
talk.politics.guns: 546
sci.med: 594
comp.sys.ibm.pc.hardware: 590
comp.os.ms-windows.misc: 591
rec.motorcycles: 598
talk.religion.misc: 377
misc.forsale: 585
alt.atheism: 480
sci.electronics: 591
comp.windows.x: 593
rec.sport.hockey: 600
rec.sport.baseball: 597
soc.religion.christian: 599
talk.politics.mideast: 564
talk.politics.misc: 465
sci.crypt: 595


Se analiza las palabras más comunes de ambos textos:

In [123]:
doc_idx_test = 6519  # índice del documento de test
doc_idx_train = 9623  # índice del documento de train

doc_test_tfidf = X_test[doc_idx_test]
doc_train_tfidf = X_train[doc_idx_train]
feature_names = np.array(tfidfvect.get_feature_names_out())

# Función para obtener las palabras más relevantes para un documento
def get_top_words(tfidf_matrix, feature_names, top_n=10):
    sorted_indices = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]
    top_indices = sorted_indices[:top_n]
    top_words = feature_names[top_indices]
    return top_words

# Palabras más comunes del documento de test y del documento más similar
top_words_test = get_top_words(doc_test_tfidf, feature_names, top_n=10)
top_words_train = get_top_words(doc_train_tfidf, feature_names, top_n=10)

print("Palabras clave en el documento de test (Índice 6519):")
print(top_words_test)

print("\nPalabras clave en el documento más similar (Índice 9623):")
print(top_words_train)



Palabras clave en el documento de test (Índice 6519):
['markov' 'nsa' 'letter' 'used' 'fortune' 'that' 'modeling' 'novel' 'the'
 'statistical']

Palabras clave en el documento más similar (Índice 9623):
['the' 'and' 'they' 'were' 'was' 'that' 'he' 'to' 'you' 'it']


Quizás la causa del error se deba a que el texto más similar tiene como palabras más comunes muchas stop words que distorcionaron la similitud de coseno.

### **Punto 3**

Primero se calculan las métricas F1-score para cada uno de los modelos, con la vectorización existente:

In [124]:
def evaluate_models(models, X_train, X_test, y_train, y_test):
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred, average='macro')
        print(f"{name:<25} -> F1-macro: {score:.3f}")


In [125]:
models = {
    "MultinomialNB (base)": MultinomialNB(),
    "ComplementNB (base)": ComplementNB()
}

evaluate_models(models, X_train, X_test, y_train, y_test)

MultinomialNB (base)      -> F1-macro: 0.585
ComplementNB (base)       -> F1-macro: 0.693


Ahora se aplicarán mejoras en la vectorización. Se analizaron los párametros existentes en TfidfVectorizer ( [link documentación](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) ). Se decidió aplicar los siguientes:

- stop_words='english': se eliminan las palabras muy comunes, llamadas stop words se sirven de conectores en los textos. La configuración es 'english', ya que los documentos del dataset estan en ese idioma.
- max_df=0.7: se ignoran los términos cuya frecuencia de aparición en los documentos sea estrictamente superior al umbral dado, en este caso aquellas que aparecen en más del 70% de los documentos.
- min_df=5: se ignoran los términos cuya frecuencia de aparición en los documentos sea estrictamente inferior al umbral especificado, en este caso, un 5%.

In [126]:
tfidfvect2 = TfidfVectorizer(
    stop_words='english',
    max_df=0.7,
    min_df=5
)

X_train2 = tfidfvect2.fit_transform(newsgroups_train.data)
X_test2 = tfidfvect2.transform(newsgroups_test.data)

models = {
    "MultinomialNB (mejorado)": MultinomialNB(),
    "ComplementNB (mejorado)": ComplementNB()
}

evaluate_models(models, X_train2, X_test2, y_train, y_test)


MultinomialNB (mejorado)  -> F1-macro: 0.650
ComplementNB (mejorado)   -> F1-macro: 0.682


En una segunda prueba se agregó:

- Sobre vectorización, sublinear_tf=True: aplica una escala logarítmica al tf.
- Sobre modelos: suavización de Laplace(alpha=0.5)


In [127]:
tfidfvect2 = TfidfVectorizer(
    stop_words='english',
    max_df=0.7,
    min_df=5,
    sublinear_tf=True
)

X_train3 = tfidfvect2.fit_transform(newsgroups_train.data)
X_test3 = tfidfvect2.transform(newsgroups_test.data)

models = {
    "MultinomialNB (mejorado 2)": MultinomialNB(alpha=0.5),
    "ComplementNB (mejorado 2)": ComplementNB(alpha=0.5)
}

evaluate_models(models, X_train3, X_test3, y_train, y_test)

MultinomialNB (mejorado 2) -> F1-macro: 0.658
ComplementNB (mejorado 2) -> F1-macro: 0.681


Con estos últimos cambios mejoró un poco la metrica para MultinomialNB.

El hecho de que la métrica para ComplementNB sea mayor que MultinomialNB puede deberse a que existe un cierto desbalance para algunas de las etiquetas y ComplementNB es adecuado para estos casos (Explicación al inicio de página: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.ComplementNB.html)